# Kannusteet, työllistymis- ja efektiivinen veroaste
Antti Tanskanen,
1.2.2022

Efektiivinen veroaste kertoo, paljonko ansaituista lisäansioista menee verojen kasvamiseen ja etuuksien pienenemisen. Työllistymisveroaste taas kertoo, mikä osuus siirtyminen työttömyydestä työhön tuomista palkkatuloista menee veroihin ja etuuksien pienenemiseen. Tulokset ovat erilaisille perheille erilaisia, erityisesti johtuen erilaisista etuuksista. Työllistymisveroaste määritellään

$$
V=1-\frac{T_2-T_1}{\Delta t},
$$

missä $T_1$ on nettotulot työttömänä sisältäen etuudet, $T_2$ on nettotulot työllistyneenä, $\Delta t$ on työstä saatava palkka. Efektiivinen marginaalivero lasketaan samalla kaavalla.

Jos esimerkiksi työttömänä käteen jää 100 e/kk, ja työllistyneenä 500 euron kuukausipalkasta jää 300 e/kk käteen verojen ja etuuksien pienentymisen jälkeen, on työllistymisveroaste 60 prosentti.

Efektiivinen marginaalivero kertoo, paljonko lisätulosta menee veroihin yms. Jos tulotaso nyt on 1500 e/kk ja siitä jää käteen etuudet mukaanlukien 1400 e/kk. Jos lisätyöllä bruttotulon saa nostettua 2000 euroon kuukaudessa, josta jää 1600 e/kk käteen (etuudet mukana), on efektiivinen veroaste 60 prosenttia.

Tarkempaa tietoa veroasteiden laskennasta löytyy esimerkiksi julkaisuista Kärkkäinen ja Tervola [2018] ja Viitamäki [2015].

## Python-kirjasto

Tässä työkirjassa esitellään Python-kirjasto fin_benefit, jonka avulla on helppo tarkastella käteen jäävien tulojen osuutta, efektiivistä marginaaliveroa ja työllistymisveroastetta. Kirjasto on melko suppea, noin 2 000 riviä koodia yhteensä.

In [1]:
# for Colab, install fin_benefits from Github
#!pip install git+https://github.com/ajtanskanen/benefits.git

In [2]:
# haetaan tarpeelliset kirjastot
import fin_benefits # määrittelee luokan, jossa on sosiaaliturvafunktiot yms
import numpy as np 

# kuvat suuremmiksi
import matplotlib.pyplot as plt

include_alv=True

plt.rcParams['figure.figsize'] = [10, 5]
imagedir='kuvat/'

/Users/anttitanskanen/opt/miniconda3/envs/py37/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


## Tyyppitapauksia ja niiden korvausasteita

Laskee työllisyysveroaste, työnteon efektiivisen marginaalivero ja käteen jäävä ansio kirjaston avulla. Valitaan ensin tyyppitapaus. Muuttuja p kertoo tarkat tiedot laskennankohteesta, selite sisältää kuvauksen siitä ja parametri tulosta kehottaa printtaamaan selite:en.

In [3]:
p,selite=fin_benefits.perheparametrit(perhetyyppi=1,kuntaryhmä=1,vuosi=2023,tulosta=True)

1 aikuinen, ei lapsia. Töissä, ei puolisoa


In [4]:
print(p)

{'tyoton': 0, 'ika': 30, 'saa_ansiopaivarahaa': 0, 't': 0, 'vakiintunutpalkka': 2500, 'perustulo': 0, 'asumismenot_toimeentulo': 537, 'asumismenot_asumistuki': 537, 'lapsia': 0, 'lapsia_paivahoidossa': 0, 'aikuisia': 1, 'veromalli': 0, 'kuntaryhma': 0, 'lapsia_kotihoidontuella': 0, 'alle3v': 0, 'ansiopvrahan_suojaosa': 1, 'ansiopvraha_lapsikorotus': 1, 'puoliso_tulot': 0, 'puoliso_tyoton': 0, 'puoliso_vakiintunutpalkka': 0, 'puoliso_saa_ansiopaivarahaa': 0, 'puoliso_tyottomyyden_kesto': 100, 'tyottomyyden_kesto': 10, 'saa_elatustukea': 0, 'elakkeella': 0, 'tyoelake': 0, 'opiskelija': 0, 'aitiysvapaalla': 0, 'isyysvapaalla': 0, 'kotihoidontuella': 0, 'puoliso_t': 0, 'asumismenot_yhdistetty': 537, 'lapsia_alle_3v': 0, 'lapsia_alle_kouluikaisia': 0, 'puoliso_elakkeella': 0, 'puoliso_opiskelija': 0, 'puoliso_tyoelake': 0, 'puoliso_aitiysvapaalla': 0, 'puoliso_isyysvapaalla': 0, 'puoliso_sairauspaivarahalla': 0, 'puoliso_kotihoidontuella': 0}


In [5]:
fin_benefits.print_examples()

Tapaus 1:
1 aikuinen, ei lapsia. Töissä, ei puolisoa

Tapaus 2:
1 aikuinen, ei lapsia. Työtön (ansiopvraha, peruste 2500 e/kk), ei puolisoa

Tapaus 3:
1 aikuinen, ei lapsia. Työtön (työmarkkinatuki), ei puolisoa

Tapaus 4:
1 aikuinen ja 1 lapsi. Töissä, ei puolisoa

Tapaus 5:
2 aikuista ja 1 lapsi. Töissä, puoliso töissä (1250 e/kk).

Tapaus 6:
2 aikuista ja 2 lasta. Töissä, puoliso töissä (1250 e/kk).

Tapaus 7:
2 aikuista ja 1 lapsi. Työtön (ansiopvraha, peruste 2500 e/kk), puoliso työtön (työmarkkinatuki)

Tapaus 8:
2 aikuista ja 1 lapsi. Töissä, puoliso töissä (1250 e/kk).

4 1
Tapaus 9:
2 aikuista ja 3 lasta. Töissä, puoliso töissä (1250 e/kk).

Tapaus 10:
1 aikuinen ja 1 lapsi. Työtön (ansiopvraha, peruste 1500 e/kk), ei puolisoa

Tapaus 11:
1 aikuinen ja 3 lasta. Työtön (ansiopvraha, peruste 2500 e/kk), ei puolisoa

Tapaus 12:
2 aikuista ja 2 lasta. Työtön (työmarkkinatuki), puoliso töissä (1250 e/kk).

Tapaus 13:
2 aikuista ja 2 lasta. Työtön (ansiopvraha, peruste 1500 e/kk), p

Tarkastetaan vielä, millaisia parametrejä p:hen on syötetty

Itse laskenta koostuu kolmesta rivistä koodia. Alustetaan objektia ben kirjastosta fin_benefits. Tämän jälkeen sitä voi kutsua laskemaan marginaalit ja plottamaan ne.

Veronmaksajat sisällyttävät kirkollisveron laskelmiin.

In [6]:
# tarkasteluvuosi
year=2023

# Luokka Benefits sisältää etuuskoodin
ben=fin_benefits.Benefits(year=year,include_kirkollisvero=True)

# Luokka Marginals sisältää marginaalien plottausfunktiot
marg=fin_benefits.Marginals(ben,year=year,incl_alv=include_alv,lang='eng')

Ja sitten lasketaan käteen jäävä tulo, efektiivinen marginaalivero, työllistymisveroaste ja osatyöstä kokoaikatyöhön siirtymän efektiivinen marginaalivero. Laskelmat tehdään 1 e/kk askelvälillä. Efektiivinen marginaalivero lasketaan 100 e/kk lisätulolla. Osatyöstä kokoaikatyöhön siirryttäessä palkka kaksinkertaistuu.

# Palkansaajan verotaulukko

Tätä voi käyttää sen verifiointiin, että palkansaajan verot ovat tässä laskettuna samat kuin Veronmaksajien taulukossa.

In [7]:
marg.verotaulukko(p)

,"kokonaistulo, e/v","palkka, e/kk",marginaali,tva,brutto,vero [%],netto
0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
1,1000.0,83.33,NaN,8.65,1000.0,8.65,913.5
2,2000.0,166.67,8.65,8.65,2000.0,8.65,1827.0
3,3000.0,250.0,8.65,8.65,3000.0,8.65,2740.5
4,4000.0,333.33,8.65,8.65,4000.0,8.65,3654.0
5,5000.0,416.67,8.65,8.65,5000.0,8.65,4567.5
6,6000.0,500.0,8.65,8.65,6000.0,8.65,5481.0
7,7000.0,583.33,8.65,8.65,7000.0,8.65,6394.5
8,8000.0,666.67,8.65,8.65,8000.0,8.65,7308.0
9,9000.0,750.0,8.65,8.65,9000.0,8.65,8221.5


In [8]:
#p,selite=fin_benefits.perheparametrit(perhetyyppi=1,kuntaryhmä=1,vuosi=2023,tulosta=True)
test=fin_benefits.Bentest()
#run_tests(p)


Running tests on 2018
työtulovähennys computed 1383.085 expected 1383.08 Passed
elaketulovahennys2018 computed 6403.200000000001 expected 6403.2 Passed
ansiotulovähennys computed 3487.2000000000003 expected 3487.2 Passed
perusvahennys2018 computed 1222.6719999999996 expected 1222.67 Passed
ylevero2018 computed 47.49999999999998 expected 47.5 Passed
valtionvero2018_tyo computed 10156.249999999998 expected 10156.25 Passed
raippavero eläke computed 409.50000000000017 expected 409.5 Passed

Running tests on 2019
työtulovähennys computed 1466.428 expected 1466.43 Passed
elaketulovahennys2019 computed 6403.200000000001 expected 6403.2 Passed
ansiotulovähennys computed 3487.2000000000003 expected 3487.2 Passed
perusvahennys2019 computed 1467.2900000000009 expected 1467.29 Passed
ylevero2019 computed 47.49999999999998 expected 47.5 Passed
valtionvero2019_tyo computed 10413.25 expected 10413.25 Passed
raippavero eläke computed 409.50000000000017 expected 409.5 Passed

Running tests on 2020
työ